# Imports and Data Loading

In [ ]:
!pip install spacy
!pip install -Iv pyspark==3.1.1
!pip install findspark

Created temporary directory: /tmp/pip-ephem-wheel-cache-yekgxq7y
Created temporary directory: /tmp/pip-req-tracker-ur2lfn77
Created requirements tracker '/tmp/pip-req-tracker-ur2lfn77'
Created temporary directory: /tmp/pip-install-8v9ttg5_
1 location(s) to search for versions of pyspark:
* https://pypi.org/simple/pyspark/
Getting page https://pypi.org/simple/pyspark/
Found index url https://pypi.org/simple
Looking up "https://pypi.org/simple/pyspark/" in the cache
Request header has "max_age" as 0, cache bypassed
Starting new HTTPS connection (1): pypi.org:443
https://pypi.org:443 "GET /simple/pyspark/ HTTP/1.1" 304 0
Analyzing links from page https://pypi.org/simple/pyspark/
  Found link https://files.pythonhosted.org/packages/65/d7/2a3b0de1178478fc00201b083d50b3d2d1affe4eac92dad3408219c5c607/pyspark-2.1.2.tar.gz#sha256=8680dcbc98e95a32f89bea70e114e139eae3543c094f063d548863522bdc4370 (from https://pypi.org/simple/pyspark/), version: 2.1.2
  Found link https://files.pythonhosted.org/pa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My\ Drive/DVA Team Kobe

/content/drive/My Drive/DVA Team Kobe


In [ ]:
# Imports
#basics
import numpy as np
import pandas as pd
import scipy 
import csv
from collections import Counter
from string import punctuation

#misc
import time

#files & data
import json
from google.colab import files

#viz
import matplotlib.pyplot as plt
import seaborn as sns 

#settings
color = sns.color_palette()
import warnings

warnings.filterwarnings('ignore')

#nlp and parsing
import nltk
import re
import pprint
from nltk import Tree
from nltk.tokenize import word_tokenize

import spacy


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import ArrayType, IntegerType



# import findspark
# findspark.init()

import math

In [ ]:
!spark-submit --version


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.1
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 11.0.10
Branch HEAD
Compiled by user ubuntu on 2021-02-22T01:33:19Z
Revision 1d550c4e90275ab418b9161925049239227f3dc9
Url https://github.com/apache/spark
Type --help for more information.


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
spark = SparkSession.builder.appName('yelp_dataset').config("spark.driver.memory", "32g").config("spark.driver.maxResultSize", "0g").getOrCreate()

In [ ]:
# RAW, UNCLEANED DATA (refer to the other notebook for how to clean, etc. Didn't want to remove anything if y'all needed it)
reviews_sk = spark.read.json('/content/drive/My Drive/DVA Team Kobe/yelp-dataset/yelp_academic_dataset_review.json')
# business_sk = spark.read.json('/content/yelp-dataset/yelp_academic_dataset_business.json')
# users_sk = spark.read.json('/content/yelp-dataset/yelp_academic_dataset_user.json')
# UNCOMMENT BELOW IF YOU NEED THEM
# tips_sk = spark.read.json('/content/yelp-dataset/yelp_academic_dataset_tip.json')
# checkin_sk = spark.read.json('/content/yelp-dataset/yelp_academic_dataset_checkin.json')
reviews_sk.summary()

# Relaxed Top n User Similarity


In [ ]:
user_review_df = reviews_sk.select("user_id", "text").groupBy("user_id").agg(concat_ws(" ", collect_list("text"))).withColumnRenamed("concat_ws( , collect_list(text))", "all_reviews")

In [ ]:
import pickle
with open('/content/drive/My Drive/DVA Team Kobe/yelp-dataset/top_reviewers.txt','rb') as f:
   my_set = pickle.load(f)

In [ ]:
top_users = ['Doz1cAtxFL_1yY0BK7VZKg',
 'VhMHpjhOkySPN6dbeZQEZQ',
 'whQlTAXTTLYg4-8h07EhGw',
 'c-O0O4RJy93KOV_aaC6WJA',
 '2BHVz9wWG2WCJ7neA3cn4w',
 'cD90zxd3NBky6BLfYPpLtg',
 'yUPfa7kzSkkmK39SvSoQJA',
 '6p8TDejFzwL4Z2QSl2_IGA',
 'shWQr-i0mdpiafoc95i9Fw',
 'EEZuNRYGVYmSxwJEov_9Wg',
 'gD6yLfZyvxHLUsFFkDgk0g',
 'egzZ0AIEBIIqgddRCl1LRg',
 'skh8lAOi2eBlX7LXlBvGAQ',
 'gDLtZ3bfj9Uoh4kVU9DVrg',
 '11H1DX-F2FsWoN3rEZxSaQ',
 'X7vozIwZtQ9e51lDbs47qw',
 'T-x348ePObp6gv_yD56c1w',
 'TVAvGCr5-wOBN1bVAeGHNA',
 '9W2P3gmJB3UHGVgzguXIQg',
 'QSNzozwTwY9UU3QNvf-3jA',
 '-GT0VWyjAdaMqT5J43jvCQ',
 '09flyL6byQaNXM7i3mqlPA',
 '-UpfPkMT7zDbR1p9CPjlGQ',
 'cmI6LKFneeyhY3j89rlrIQ',
 'QaNQb67Hv-kZLHtVJ96h8w',
 'n1VYjLuFOhs_SnYZ4AD62w',
 'a4cUWhGUbAj8o2cLZq89hg',
 'vvoejvk5g_Ec4K9ZK4sQvw',
 'zsK4_0uIOWesa6YQFQHRkQ',
 'OV6JZ1M5ov65F8PWQzhG-Q',
 'IPPBj-9PxETzKlnHy6uU4A',
 'n8sJ815w2UgksuvvxJxHxQ',
 's0qsZ0_GtpuR6hfG8g1u6w',
 'Yny4tKfTnLHjMJ75dXPQ-Q',
 'zua3EQdxfBehVmwALcS3Dg',
 '58bJVgt2JyI6LVAcCg5r7g',
 'JYlq5s44trewHH-FmuhjaQ',
 'sobIRdjEjHtD2oKHTBk-WQ',
 'pdUOlB57aYRhhrY804DfLw',
 'rE0GgCcSh_JbeZfwu3fQ7w',
 '_0cWueK8TnVQgzDAHYzmqA',
 'i_PtC8hn_zTgjh1jiTU2cg',
 'lObqYbgzg-apheA-GtBdIg',
 'DUEssU5xTOTzLpMasbjzBw',
 'S7jEEZVw3EJ6Guo3IiKCUA',
 'Tu4RVtzluq5sf5UkwkvOaQ',
 'hHTi617dits3BjVnffjiwQ',
 '2p6nhUVYlULxA4AsKpJZ1A',
 'j5PY-M9wjWXpqhvp_k6S9g',
 'sD4VkBiiy_ABvwkc1PPxrA',
 's4NgvdIfBH3UQdccWCyARg']

user_review_filtered_df = user_review_df.filter(user_review_df.user_id.isin(top_users))

In [ ]:
user_review_filtered_df.count()

51

In [ ]:
# df2 = user_review_df.withColumn('length', length(user_review_df.user_id))
# df2.show(1)

In [ ]:
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        # 4
        if(token.pos_ in pos_tag):
            result.append(token.text)
                
    return result # 5

In [ ]:
def get_average_vector(text, n):

  keywords = get_hotwords(text)

  top_keywords = [(x[0]) for x in Counter(keywords).most_common(n)]
  # print(' '.join(hashtags))
  vectors = np.array([nlp(keyword).vector for keyword in top_keywords])
  vector = np.mean(vectors, axis=1)

  return vector

  # for word, vector in keyword_vectors.items():
  #   print(word)
  #   print(vector)
  #   print('\n')

In [ ]:
embedding_dict = {}

In [ ]:
# with open('/content/drive/My Drive/DVA Team Kobe/yelp-dataset/user_embeddings.json', 'r') as f:
#   embedding_dict = json.load(f)
# embedding_dict

In [ ]:
rows = user_review_filtered_df.rdd.toLocalIterator()
rows

<generator object _local_iterator_from_socket.<locals>.PyLocalIterable.__iter__ at 0x7f5b68214dd0>

In [ ]:
i = 0
for row in rows:
  i += 1
  print(i)
  user_id = row[0]
  text = row[1]
  nlp.max_length = len(text) + 100
  print(user_id)
  print(len(text))
  embedding_dict[user_id] = get_average_vector(text, 20).tolist()
  print(embedding_dict[user_id])
  print('\n')

1
QaNQb67Hv-kZLHtVJ96h8w
6762
[0.044468265026807785, 0.06947875022888184, 0.05058904364705086, 0.04411081597208977, 0.07107695192098618, 0.05974708870053291, 0.04782283306121826, 0.0623808391392231, 0.0430048406124115, 0.0741322785615921, 0.07170452177524567, 0.03368326649069786, 0.06190994381904602, 0.07015404850244522, 0.02406400255858898, 0.03969603776931763, 0.05348549410700798, 0.04178548976778984, 0.049523163586854935, 0.024930166080594063]


2
zua3EQdxfBehVmwALcS3Dg
5059
[0.05058904364705086, 0.061858583241701126, 0.0481337308883667, 0.0568704791367054, 0.044468265026807785, 0.0515744686126709, 0.07170452177524567, 0.05102381110191345, 0.05563497543334961, 0.05799838900566101, 0.02533380128443241, 0.06209378316998482, 0.051255881786346436, 0.04869483783841133, 0.026244571432471275, 0.0205413606017828, 0.045526061207056046, 0.021388361230492592, 0.04411081597208977, 0.05924452468752861]


3
i_PtC8hn_zTgjh1jiTU2cg
3585
[0.044468265026807785, 0.04782283306121826, 0.0439034700393676

In [ ]:
import json

with open('/content/drive/My Drive/DVA Team Kobe/yelp-dataset/user_embeddings.json', 'w') as f:
    json.dump(embedding_dict, f)

In [ ]:
# for user in embedding_dict:
#   embedding_dict[user] = embedding_dict[user].tolist() 

In [ ]:
from scipy import spatial

similarity_dict = {}
for key, value in embedding_dict.items():
  similarity_dict[key] = 1 - spatial.distance.cosine(value, embedding_dict['s4NgvdIfBH3UQdccWCyARg'])

similarity_dict

{'-GT0VWyjAdaMqT5J43jvCQ': 0.8959329078578347,
 '-UpfPkMT7zDbR1p9CPjlGQ': 0.8381858028350193,
 '09flyL6byQaNXM7i3mqlPA': 0.9091749835062369,
 '11H1DX-F2FsWoN3rEZxSaQ': 0.8661781479467043,
 '2BHVz9wWG2WCJ7neA3cn4w': 0.8534487237524433,
 '2p6nhUVYlULxA4AsKpJZ1A': 0.8905552365208247,
 '58bJVgt2JyI6LVAcCg5r7g': 0.8693958026420622,
 '6p8TDejFzwL4Z2QSl2_IGA': 0.8696067711206253,
 '9W2P3gmJB3UHGVgzguXIQg': 0.861703546713536,
 'DUEssU5xTOTzLpMasbjzBw': 0.9094777672206393,
 'Doz1cAtxFL_1yY0BK7VZKg': 0.9482174183652212,
 'EEZuNRYGVYmSxwJEov_9Wg': 0.9167187111491407,
 'IPPBj-9PxETzKlnHy6uU4A': 0.8734155293137652,
 'JYlq5s44trewHH-FmuhjaQ': 0.8822279640715003,
 'OV6JZ1M5ov65F8PWQzhG-Q': 0.8635132054789256,
 'QSNzozwTwY9UU3QNvf-3jA': 0.8514209692730635,
 'QaNQb67Hv-kZLHtVJ96h8w': 0.9288151327603632,
 'S7jEEZVw3EJ6Guo3IiKCUA': 0.8591914982416539,
 'T-x348ePObp6gv_yD56c1w': 0.8506262199403428,
 'TVAvGCr5-wOBN1bVAeGHNA': 0.8335407610991097,
 'Tu4RVtzluq5sf5UkwkvOaQ': 0.9033889340760864,
 'VhMHpjhOkySP

In [ ]:
with open('/content/drive/My Drive/DVA Team Kobe/yelp-dataset/user_s4NgvdIfBH3UQdccWCyARg_textual_similarity.json', 'w') as f:
    json.dump(embedding_dict, f)

# Top Recommended Restaurant Hotwords

In [ ]:
restaurant_review_df = reviews_sk.select("business_id", "text").groupBy("business_id").agg(concat_ws(" ", collect_list("text"))).withColumnRenamed("concat_ws( , collect_list(text))", "all_reviews")

In [ ]:
with open('/content/drive/My Drive/DVA Team Kobe/yelp-dataset/recommendations_for_selected_users.json', 'r') as f:
  rec_dict = json.load(f)
top_restaurants = []
for user in rec_dict:
  top_restaurants += rec_dict[user]["recommended_business_id"]
top_restaurants

['6PLX0UeiErFh9K8ni-xoCA',
 'Pey3Wjl1aNExX-TrZsUvzw',
 'Be9cTYcve9MvIDgfFrDyDQ',
 'nSy2ozEBcEcAVoQKmvY6Tw',
 '3qlgCxdeBp3MDavPrFIuvw',
 'tyKwmkBgtYgDYoSv104DNw',
 'LBYB2cnASK8vbSnfW851uw',
 '66DY7zz0BHGmnhHlXy2IrA',
 'LhVtjCT3U89MOCQLAtxNog',
 'ooAWQE5DYHv3rsXrM0IvOg',
 'q9AVhmfXfFgIhL6nZgr9ZQ',
 '7895YHS5qGu03R-C6aYg9g',
 'AiFVqAFneSse0hExrzQTEg',
 'tWG4oTk1qgRhRPmkpenA1A',
 '3KP1v_NctaJWNIX_YsYdKQ',
 'J12N556sezpr6BfPYOslFg',
 'gGvNgShksetPoimyKV8I9Q',
 '0HuvhjQoUSo37YaJjelmrw',
 'P3TJvj2UfAOyQ61_qChlAQ',
 'tcn5rNJcDMoLdbROJjCvJQ',
 '1OoI1Jjtmn3-FfyjjBP3yw',
 'LCuqEMs-2MiR7ilr66o_Kw',
 'jC3PI3qNeZmnOG3xmI6USQ',
 'Yebpocqt88Bt542T-T7yQA',
 '6AtGmd-i19wpTZLqv6x1_A',
 '9tC6Q9lcTbjd-2fNySNbDw',
 'mQdmfIcPOI6QJJaBrSmBUg',
 'GfSUPCem6dslmRpnMi3O_A',
 'vmqE8aRZ18ChH0-tL8L1aQ',
 '8XB37Y7YMjrpltgN09FBXg',
 'Jtt-W-mSOJ6igRGjxHZZ7Q',
 'WU8ClbOZm6E5kuwe709mkQ',
 '-EzfZm6rTohZdD9tfQaMyA',
 'jscwVHaMMA0A3IkgHUAatQ',
 'e8itvuHd8cfA7MuAZRrqqQ',
 'zylYdVSB_KCGsB-Vunncsw',
 'ieovrBlSOXRZUHCb6qMmCg',
 

In [ ]:
restaurant_review_df = restaurant_review_df.filter(restaurant_review_df.business_id.isin(top_restaurants))

In [ ]:
restaurant_review_df.count()

220

In [ ]:
# df2 = user_review_df.withColumn('length', length(user_review_df.user_id))
# df2.show(1)

In [ ]:
def get_hotwords(text, n):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        # 4
        if(token.pos_ in pos_tag):
            result.append(token.text)
                
    top_keywords = [(x[0]) for x in Counter(result).most_common(n)]
    return top_keywords

In [ ]:
def get_average_vector(text, n):

  keywords = get_hotwords(text)

  top_keywords = [(x[0]) for x in Counter(keywords).most_common(n)]
  # print(' '.join(hashtags))
  vectors = np.array([nlp(keyword).vector for keyword in top_keywords])
  vector = np.mean(vectors, axis=1)

  return vector

  # for word, vector in keyword_vectors.items():
  #   print(word)
  #   print(vector)
  #   print('\n')

In [ ]:
words_dict = {}

In [ ]:
# with open('/content/drive/My Drive/DVA Team Kobe/yelp-dataset/user_embeddings.json', 'r') as f:
#   embedding_dict = json.load(f)
# embedding_dict

{'0V_C2aOTDKuR8CO3P3RvJw': [0.04411081597208977,
  0.06947875022888184,
  0.044468265026807785,
  0.04782283306121826,
  0.024930166080594063,
  0.038725320249795914,
  0.017111921682953835,
  0.0724467858672142,
  0.0481337308883667,
  0.024662330746650696,
  0.051117878407239914,
  0.036246467381715775,
  0.053581759333610535,
  0.07015404850244522,
  0.02533380128443241,
  0.06832128018140793,
  0.04379599913954735,
  0.032940853387117386,
  0.050178248435258865,
  0.03969603776931763],
 '1IqicHIwRvjl83yGVhIooA': [0.044468265026807785,
  0.06947875022888184,
  0.04411081597208977,
  0.026218563318252563,
  0.024930166080594063,
  0.07835084944963455,
  0.07015404850244522,
  0.04782283306121826,
  0.061858583241701126,
  0.05058904364705086,
  0.033340394496917725,
  0.05129288509488106,
  0.03912550210952759,
  0.04607267677783966,
  0.07648837566375732,
  0.042586397379636765,
  0.026391467079520226,
  0.03432770445942879,
  0.04024621471762657,
  0.026747852563858032],
 '2-2IewhT

In [ ]:
rows = restaurant_review_df.rdd.toLocalIterator()
rows

<generator object _local_iterator_from_socket.<locals>.PyLocalIterable.__iter__ at 0x7f303ecffcd0>

In [ ]:
i = 0
for row in rows:
  i += 1
  print(i)
  business_id = row[0]
  text = row[1]
  nlp.max_length = len(text) + 100
  print(buisness_id)
  print(len(text))
  words_dict[business_id] = get_hotwords(text, 20)
  print(words_dict[business_id])
  print('\n')

1
Z2qMwUhnGt_2pA9uQbS7Uw
28592
['pizza', 'chicago', 'good', 'place', 'time', 'order', 'service', 'food', 'great', 'style', 'deep', 'dish', 'crust', 'east', 'sauce', 'minutes', 'wings', 'best', 'delivery', 'nice']


2
Z2qMwUhnGt_2pA9uQbS7Uw
54629
['food', 'greek', 'good', 'place', 'gyro', 'potatoes', 'service', 'chicken', 'meat', 'pita', 'lunch', 'great', 'salad', 'huge', 'mykonos', 'time', 'little', 'best', 'friendly', 'portions']


3
Z2qMwUhnGt_2pA9uQbS7Uw
63019
['lobster', 'food', 'red', 'service', 'good', 'time', 'location', 'great', 'order', 'shrimp', 'server', 'waitress', 'experience', 'place', 'bad', 'biscuits', 'restaurant', 'salad', 'people', 'table']


4
Z2qMwUhnGt_2pA9uQbS7Uw
155793
['food', 'great', 'good', 'place', 'irish', 'pub', 'service', 'bar', 'beer', 'menu', 'sandwich', 'fish', 'time', 'dog', 'drinks', 'chips', 'blind', 'atmosphere', 'chicken', 'nice']


5
Z2qMwUhnGt_2pA9uQbS7Uw
9684
['coffee', 'kaldi', 'emory', 'latte', 'good', 'place', 'sandwich', 'decaf', 'campus',

In [ ]:
import json

with open('/content/drive/My Drive/DVA Team Kobe/yelp-dataset/restaurant_hotwords.json', 'w') as f:
    json.dump(words_dict, f)

In [ ]:
# for user in embedding_dict:
#   embedding_dict[user] = embedding_dict[user].tolist() 